In [1]:

import pandas as pd
import requests
import json
from datetime import datetime
import awswrangler as wr
from calendar import monthrange
import warnings
warnings.filterwarnings("ignore")
import random
import numpy as np
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd


# Cpo contract address

In [2]:
bscan_api_key = "QFJJI57H6QYKZUWKV5W3K8HFRBB5CX6RAX"

# On y trouve les rewards qui sont envoyés tous les jours à l'addresse wallet_stacking_rewards par l'intermédiaire du contrat 0xea395DFaFEd39924988b475f2Ca7f4C72655203A
tax_for_stacking_reward_address ="0x6cfAB07606Ee22A4c0B75cc3f2C88Ffd37C9d0F8"

# Quand je stack mes stacking rewards, ils sont transféré depuis cette addresse, jusqu'à l'addresse du stacking contract
# Addresse qui contient les stacking rewards
wallet_stacking_rewards = "0x60e75cb4f51bfac38c845d8c240d6a6b03dac2a5"

# Le smart contract cryptopolis. 
# Il a une méthode distribute reward qui va trensférer 
stacking_contract = "0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334"


my_bsc = "0x73542beb26ca10c9f7ccbb6826f3b77d7014e298"


# Quand on withdraw les stacking rewards c'est par l'intermédiaire du wallet_stacking_rewards.
unstack_and_restack_address= "0x54fab6b183269bcc3a2767957492ce4196e3e543"


# Retrieve data

In [3]:
def get_last_block(_df):
    return _df["blockNumber"].sort_values().values.tolist()[-1]

def get_data_on_all_block(method, bsc_address):
    chunk_empty = False
    data = []
    initial_block = 0
    loop_nb = 1
    while chunk_empty is False:
        print(f"Loop number {loop_nb}")
        loop_nb+=1
        chunk = method(bsc_address, initial_block)
        print(chunk.shape)
        data.append(chunk)
        if len(chunk)<=2:
            chunk_empty = True
        initial_block = get_last_block(chunk)
    return pd.concat(data).drop_duplicates(keep="first")

def get_transaction_from_block(bsc_address:str, block_nb:int):
    url = f"""https://api.bscscan.com/api?module=account\
&action=txlist\
&address={bsc_address}\
&startblock={block_nb}\
&endblock=99999999\
&sort=asc\
&apikey={bscan_api_key}"""
    return pd.DataFrame(json.loads(requests.get(url).content)["result"])

def get_token_transfered_from_and_to_an_address_from_block(bsc_address:str, block_nb:int):
    url  = f"""https://api.bscscan.com/api?module=account\
&action=tokentx\
&address={bsc_address}\
&startblock={block_nb}\
&endblock=999999999\
&sort=asc\
&apikey={bscan_api_key}"""
    return pd.DataFrame(json.loads(requests.get(url).content)["result"])


In [4]:
all_token_transfered_from_and_to_stacking_contract = get_data_on_all_block(get_token_transfered_from_and_to_an_address_from_block, bsc_address=stacking_contract)

Loop number 1
(10000, 19)
Loop number 2
(10000, 19)
Loop number 3
(10000, 19)
Loop number 4
(324, 19)
Loop number 5
(2, 19)


In [8]:

wr.s3.to_parquet(df=all_token_transfered_from_and_to_stacking_contract, dataset=True, path="s3://crypto-datahub/cryptopolis/token_transfer" ,mode="overwrite", database="cryptopolis", table="token_transfer")


{'paths': ['s3://crypto-datahub/cryptopolis/token_transfer/07fa0030e53d40f99da2dd48487705a0.snappy.parquet'],
 'partitions_values': {}}

In [5]:
all_stacking_contract_transaction = get_data_on_all_block(get_transaction_from_block, bsc_address=stacking_contract)

Loop number 1
(10000, 18)
Loop number 2
(10000, 18)
Loop number 3
(10000, 18)
Loop number 4
(1842, 18)
Loop number 5
(1, 18)


In [7]:
all_token_transfered_from_and_to_stacking_contract["value"] = pd.to_numeric(all_token_transfered_from_and_to_stacking_contract["value"], errors="coerce")/1e18

In [8]:
all_token_transfered_from_and_to_stacking_contract["datetime"] = pd.to_datetime(all_token_transfered_from_and_to_stacking_contract["timeStamp"], unit="s")
all_stacking_contract_transaction["datetime"] = pd.to_datetime(all_stacking_contract_transaction["timeStamp"], unit="s")

In [386]:
# Je retrouve toutes les transactions que j'ai effectué avec le contrat publique cryptopolis.
my_transaction_with_cpo_contract = all_stacking_contract_transaction[all_stacking_contract_transaction["from"]==my_bsc]
my_transaction_with_cpo_contract_hash = my_transaction_with_cpo_contract["hash"].values.tolist()

In [47]:
# Retrieve all stackers
list_stackers = list(all_token_transfered_from_and_to_stacking_contract["from"].unique())
list_sellers = list(all_token_transfered_from_and_to_stacking_contract["to"].unique())
# list_stackers_that_never_withdraw = list(all_token_transfered_from_and_to_stacking_contract[~all_token_transfered_from_and_to_stacking_contract["to"].isin(list_stackers)]["from"].unique())

In [48]:
list_sellers

['...']

In [389]:
df_token_that_I_stacked = all_token_transfered_from_and_to_stacking_contract[all_token_transfered_from_and_to_stacking_contract["from"]==my_bsc]

In [390]:
# Je récupère tous les transfers de token depuis le wallet qui contient les stacking rewards
stacking_rewards_token_transfer_from_stacking_rewards_wallet = all_token_transfered_from_and_to_stacking_contract[all_token_transfered_from_and_to_stacking_contract["from"]==wallet_stacking_rewards]
# Puis je fais le join avec les transactions que j'ai effectué. De cette manière je vais retrouver toutes les transactions que j'ai effectué sur le stacking contract qui ont pour effet de transférer des token depuis le wallet de reward vers le smart contract
df_my_stacking_rewards = stacking_rewards_token_transfer_from_stacking_rewards_wallet[stacking_rewards_token_transfer_from_stacking_rewards_wallet["hash"].isin(my_transaction_with_cpo_contract_hash)]

In [391]:
my_stacking_rewards = df_my_stacking_rewards[["datetime","value"]]
token_that_I_stacked = df_token_that_I_stacked[["datetime", "value"]]


In [409]:
my_token_stack_evolution = pd.concat([my_stacking_rewards, token_that_I_stacked]).sort_values("datetime")
my_token_stack_evolution["date"] = my_token_stack_evolution["datetime"].dt.date
my_token_stack_evolution["cumsum"]=my_token_stack_evolution["value"].cumsum()

In [478]:
tier_calculus_df = my_token_stack_evolution.set_index("datetime").resample("1M", label="right").agg({"cumsum":"last", "date":min})
tier_calculus_df.reset_index(inplace=True)

In [507]:
tier_calculus_df[["booster_pack_tier_1", "booster_pack_tier_2", "booster_pack_tier_3"]] =tier_calculus_df.apply(compute_nft_booster_pack, axis=1, result_type="expand")
print(tier_calculus_df)

    datetime        cumsum        date  booster_pack_tier_1  \
0 2021-10-31   6247.416972  2021-10-25                  1.0   
1 2021-11-30   7579.844366  2021-11-01                  0.0   
2 2021-12-31  18352.479980  2021-12-01                  0.0   
3 2022-01-31  18394.650232  2022-01-01                  0.0   

   booster_pack_tier_2  booster_pack_tier_3  
0                  0.0                  0.0  
1                  1.0                  0.0  
2                  3.0                  0.0  
3                  3.0                  0.0  


In [491]:
def compute_nft_booster_pack(row):
    date = row["datetime"]
    min_date = row["date"]
    nb_jour = monthrange(date.year, date.month)[1]
    ratio = ((nb_jour-min_date.day)+1)/nb_jour
    relative_token_amount = ratio * row["cumsum"]
    reward = {"booster_pack_tier_1":0,"booster_pack_tier_2":0,"booster_pack_tier_3":0}
    if relative_token_amount < 6000:
        reward["booster_pack_tier_1"] = relative_token_amount // 750
    if relative_token_amount < 30000:
        reward["booster_pack_tier_2"] = relative_token_amount // 6000
    if relative_token_amount >= 30000:
        reward["booster_pack_tier_3"] = relative_token_amount // 30000
    return reward

# From csv export

In [50]:
df = pd.read_csv("transaction-batch1.csv")

In [54]:
df["datetime"] = pd.to_datetime(df["UnixTimestamp"])

In [55]:
df["datetime"]

0xbd3b084eb1389755217f75d592ed24fb0c204e7bcc617c9114f59b0228d3ad86   2021-10-23 02:08:19
0xe5ad17e6484b251c77e552f9aec7ebe98214c06cf747a0f0ad7bdb3f7d9a29b1   2021-10-23 02:12:01
0xa612ab1a5d209aa6483eaa51b13f43f212a5216c7b70a5a2cbc4db7b5991f0aa   2021-10-23 02:13:10
0x101d26173e8b38229d875a15a9496ac9d2d70e509fc90f348e575d47020b3c6e   2021-10-23 05:51:41
0x0b92957cd3717ff5742966dc24242960d9c10ed07eb12a22d0e49f2184cbdaf4   2021-10-23 05:52:32
                                                                             ...        
0xa2b5fea2b39d8098a252e62dea21fc36aaa0907b195049c91c337f2fd455b7bf   2021-11-01 14:26:34
0xae1af1b56b3ca73777f87718f589a45607c34e44aab99542b1d7b147cbf211fb   2021-11-01 14:27:52
0x8e73d1fe66be0ac5ffe10715b7576e414994f28d458bc23297e5bc7fbd492283   2021-11-01 14:32:59
0xd18e442d55835a7dfe5c11c401d53eb039644e3b969c95879f065ef8a098fa22   2021-11-01 14:35:29
0x5b2e3fed128efc647382c4effac57879662eea9bc7e485c94d9fdc330ea4faaa   2021-11-01 14:35:38
Name: datetime, Lengt

In [7]:
import os
csv_files = [x for x in os.listdir(".") if ".csv" in x]
for file in csv_files:
    df = pd.read_csv(file)
    df.columns = ["t_"+x.lower() for x in df.columns]
    dtype={}
    for col in ["t_txhash", "t_from","t_to", "t_contractaddress", "t_method"]:
        dtype[col]="string"
    dtype["datetime"]="timestamp"
    wr.s3.to_parquet(df=df, dataset=True, path="s3://crypto-datahub/cryptopolis/transaction" ,database="cryptopolis", table="transaction", dtype=dtype)
    

In [9]:
df = wr.athena.read_sql_query(
    """SELECT * from "transaction" A LEFT JOIN "token_transfer" B ON A.t_txhash=B.hash"""
    , database="cryptopolis")

In [10]:
df["t_method"].value_counts().head(5)

Stake Rewards       18139
Stake               12218
Withdraw Rewards     3263
Unstake              1280
Withdraw             1120
Name: t_method, dtype: Int64

In [209]:
df["t_datetime"]= pd.to_datetime(df["t_datetime"])
df.drop_duplicates("t_txhash", inplace=True)

In [210]:
# Retrieve all stacker address
stackers = list(df[df["t_method"]=="Stake"]["t_from"].unique())
unstackers = list(df[df["t_method"]=="Unstake"]["t_from"].unique())
print(f"Il y a {len(stackers)} stackers")
print(f"Il y a {len(unstackers)} unstackers")

Il y a 4088 stackers
Il y a 1114 unstackers


In [285]:
random.shuffle(stackers)
list_month = pd.date_range(start="2021-11-01", end="2022-04-01", freq="MS")
for stacker in stackers:
    
    df_stacker = df[df["t_from"]==stacker]
    df_stacker = df_stacker[df_stacker["t_status"].isna()]
    df_stacker = df_stacker[["t_method","t_datetime","value"]]
    df_stacker.sort_values("t_datetime", inplace=True)
    df_stacker["value"] = df_stacker["value"].astype(float)/10e18
    df_stacker.reset_index(inplace=True)
    
    
    
    
    
    df_stacker.loc[df_stacker.t_method=="Withdraw", 'value'] = df_stacker.loc[df_stacker.t_method=="Withdraw", 'value'] *-1
    df_stacker["wallet_value"] = df_stacker["value"].cumsum()
    
    # S'il unstack mais qu'il withdraw pas alors on ne peut pas le traiter
    df_stacker_withdraw = df_stacker[df_stacker.t_method=="Unstake"]
    if len(df_stacker_withdraw):
        last_withdraw_index = df_stacker_withdraw.index.tolist()[-1]
        df_stacker_after_last_withdraw = df_stacker.loc[last_withdraw_index:]
        # Il y t il un withdraw apres le last usntack?
        if len(df_stacker_after_last_withdraw[df_stacker_after_last_withdraw["t_method"]=="Withdraw"])==0:
            print(df_stacker)
    
    continue
    first_transaction_in_october = df_stacker["t_datetime"].min().month==10
    
    df_stacker["month"] = df_stacker["t_datetime"].dt.strftime("%Y-%m")
    agg={}
    agg["t_datetime"] = ["first"]
    agg["wallet_value"] = ["last"]
    
    print(df_stacker[["t_datetime","value","wallet_value","t_method"]])
    df_stacker_by_month = df_stacker.groupby("month").agg(agg)
    df_stacker_by_month.reset_index(inplace=True)
    df_stacker_by_month.columns = df_stacker_by_month.columns.map('|'.join).str.strip('|')
    print(df_stacker_by_month)
    if not first_transaction_in_october:
        # On corrige le premier last wallet
        # On retrouve la date de la premiere transa du mois (qui est soit Novembre, Decembre, Janvier, ..)
        print(df_stacker_by_month.loc[0,"t_datetime|first"])

        


    
    break
    
    # If withdraw transactions found, we keep transaction after the last withdraw
    # withdraw_index = df_stacker.loc[df_stacker.t_method=="Withdraw"].index.tolist()
    # if len(withdraw_index)>0:
    #     df_stacker = df_stacker.iloc[withdraw_index[-1]:, :]
    #     df_stacker.reset_index(inplace=True)
        
    # # Retrieve first transaction
    # first_transaction = df_stacker.loc[0,:]
    # break
    
    
    
    for month in list_month:
        # Retrieve first 
        if month ==list_month[0]:
            print(month)
        # Retrieve transaction of the month
        month_transaction = df_stacker[(df_stacker["t_datetime"]>=month) & (df_stacker["t_datetime"]<=month + MonthEnd(0))]
        print(month_transaction[["t_datetime","value","wallet_value","t_method"]])
        
        
    
    

    break
    
    
    
    # Retrieve month of the transaction (2021-11)
    df_stacker["month"] = df_stacker["t_datetime"].dt.strftime("%Y-%m")
    
    # Retrieve nb of day on the month of the transaction (2021-10-25 12:15:33 => 31)
    df_stacker['days_in_month'] = df_stacker['t_datetime'].dt.days_in_month
    
    # Retrieve the day if month of the transaction (2021-10-25 12:15:33 => 25)
    df_stacker['day_of_month'] = df_stacker['t_datetime'].dt.day
    print("INDEX DONNEES MOIS OCTOBRE")
    print(df_stacker[df_stacker.month=="2021-10"].index)
    
    print("DONNEES AVANT DROP OCTOBRE")
    print(df_stacker[["t_datetime","value","days_in_month", "day_of_month"]])
    df_stacker = df_stacker.drop(df_stacker[df_stacker.month=="2021-10"].index)

    agg={}
    agg["t_datetime"] = ["first","last"]
    agg["cumsum"] = ["last"]
    print("DONNEES APRES GROUPBY")
    print(df_stacker.groupby("month").agg(agg))
    print("\n"*5)

    
    
    


        
        


    

   index t_method          t_datetime       value  wallet_value
0  11236    Stake 2021-11-26 16:47:08  127.637073    127.637073
1  27680  Unstake 2021-12-04 18:06:17         NaN           NaN
   index t_method          t_datetime       value  wallet_value
0  26219    Stake 2021-11-21 15:47:12  351.803762    351.803762
1  33191    Stake 2021-11-30 13:17:51    5.861242    357.665005
2  33196  Unstake 2021-11-30 13:22:56         NaN           NaN
    index       t_method          t_datetime      value  wallet_value
0    1467          Stake 2021-11-01 19:33:20  76.785796     76.785796
1    8612  Stake Rewards 2021-11-02 18:59:27   1.181777     77.967572
2    8652          Stake 2021-11-02 19:29:11  82.170215    160.137788
3   16631          Stake 2021-11-04 15:10:56   2.392736    162.530524
4   31366  Stake Rewards 2021-11-05 17:03:00   4.542551    167.073075
5    2839  Stake Rewards 2021-11-07 18:33:59  13.577034    180.650109
6   19470  Stake Rewards 2021-11-09 19:32:19   4.373352    185

KeyboardInterrupt: 

In [5]:
all_token_transfered_from_and_to_stacking_contract

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,12013383,1634968301,0x101d26173e8b38229d875a15a9496ac9d2d70e509fc9...,18,0x67f0fe04dac546ca455170040e4f1aec02807c248536...,0x992784ddc63044acfcdca5a40618783ee131dfa8,0xea395dfafed39924988b475f2ca7f4c72655203a,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,1000000000000000000,Cryptopolis,CPO,18,116,425038,5000000000,258646,15635545,deprecated,2174556
1,12013557,1634968823,0x14d5deeea0bdedc985621068d19d9a2a75a9ff026261...,23,0x1876a7d19350e0ae3906e6469e0a49523d6f82dea20a...,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,0xea395dfafed39924988b475f2ca7f4c72655203a,0x992784ddc63044acfcdca5a40618783ee131dfa8,1000000000000000000,Cryptopolis,CPO,18,316,162082,5000000000,52637,35963853,deprecated,2174382
2,12014624,1634972049,0x3fb87b6de0bf16f55dbf53fab785c9654b6ba9289f56...,26,0xc44a05e142e2300f538dc4da0e3464e340e176218bdf...,0x992784ddc63044acfcdca5a40618783ee131dfa8,0xea395dfafed39924988b475f2ca7f4c72655203a,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,1000000000000000000,Cryptopolis,CPO,18,165,425038,5000000000,258646,22657840,deprecated,2173315
3,12014708,1634972302,0x521dc41d4256a5fd9899bfdefc1734e17998bd96ff0e...,540,0x2ec2a4db50b5d15a3a947a97406b75914b6e2b9f4f22...,0xe4f5615c9dd34f514c85c4b1ffa0b893753fe9b4,0xea395dfafed39924988b475f2ca7f4c72655203a,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,1000000000000000000,Cryptopolis,CPO,18,94,362868,5000000000,217846,11000604,deprecated,2173231
4,12014727,1634972359,0x37692d396029e50e23a5b0c215b317e8831db150f021...,28,0x7d8abef8de0dee2d465dcbc44fb4a134a6779c679e4f...,0x992784ddc63044acfcdca5a40618783ee131dfa8,0xea395dfafed39924988b475f2ca7f4c72655203a,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,1000000000000000000,Cryptopolis,CPO,18,171,261601,5000000000,136390,23616152,deprecated,2173212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,14187654,1641650550,0xba21ab12dd475ba6c9657c8646e81df7817147052cf6...,53,0x6370de57cd3363b7cd19dab00a1cf70f6589cf0ff741...,0x60e75cb4f51bfac38c845d8c240d6a6b03dac2a5,0xea395dfafed39924988b475f2ca7f4c72655203a,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,11220836236944885721,Cryptopolis,CPO,18,182,164052,5000000000,102364,28857881,deprecated,297
322,14187682,1641650634,0xdf071aa84fa26968d3b20de4f9d90c438dd05546f69b...,1127,0x1cd819df96d49a9bc8e6fa9267e6a3d87e286d6e1b28...,0x3305b48eb48e2dda716c7adcc4d65c6d042c344c,0xea395dfafed39924988b475f2ca7f4c72655203a,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,4879180892721270718232,Cryptopolis,CPO,18,87,215959,5000000000,106438,11724597,deprecated,269
323,14187682,1641650634,0xdf071aa84fa26968d3b20de4f9d90c438dd05546f69b...,1127,0x1cd819df96d49a9bc8e6fa9267e6a3d87e286d6e1b28...,0x60e75cb4f51bfac38c845d8c240d6a6b03dac2a5,0xea395dfafed39924988b475f2ca7f4c72655203a,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,170769703889598379595,Cryptopolis,CPO,18,87,215959,5000000000,106438,11724597,deprecated,269
0,14187682,1641650634,0xdf071aa84fa26968d3b20de4f9d90c438dd05546f69b...,1127,0x1cd819df96d49a9bc8e6fa9267e6a3d87e286d6e1b28...,0x3305b48eb48e2dda716c7adcc4d65c6d042c344c,0xea395dfafed39924988b475f2ca7f4c72655203a,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,4879180892721270718232,Cryptopolis,CPO,18,87,215959,5000000000,106438,11724597,deprecated,270


In [6]:
all_token_transfered_from_and_to_stacking_contract["datetime"] = pd.to_datetime(all_token_transfered_from_and_to_stacking_contract["timeStamp"], unit="s")

In [9]:
all_token_transfered_from_and_to_stacking_contract["day"] = all_token_transfered_from_and_to_stacking_contract.datetime.dt.strftime("%Y-%m-%d")

In [151]:
df=all_token_transfered_from_and_to_stacking_contract[["datetime","from","to","day"]]

# Expériences

In [182]:
def clean_s3():
    wr.s3.delete_objects("s3://crypto-datahub/test_folder")
def clean_glue_table():
    wr.catalog.delete_table_if_exists(table="test_table",database="test")

def clean():
    clean_s3()
    clean_glue_table()

def list_s3():
    return wr.s3.list_objects("s3://crypto-datahub/test_folder")

In [155]:
df.sample(5, random_state=1)

,datetime,from,to,day
6683,2021-12-23 05:43:47,0x32fcf06e259601976e0df7969d43bd640ef57cfb,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,2021-12-23
4499,2021-11-01 12:09:07,0xe5ea6ee251a109147d166c356bdb76d301a4259c,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,2021-11-01
4312,2021-10-31 22:00:07,0x086f411827b12e5e69f0b3123901a2559a25f74e,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,2021-10-31
6848,2021-11-04 00:54:36,0x60e75cb4f51bfac38c845d8c240d6a6b03dac2a5,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,2021-11-04
6587,2021-12-22 19:43:26,0x60e75cb4f51bfac38c845d8c240d6a6b03dac2a5,0x8e3cfca4995d2ec53e1001707cd6f3d58a32b334,2021-12-22


In [156]:
df_2021_11_01 = df[df["day"]=="2021-11-01"]
df_2021_11_01_02 = df[(df["day"]=="2021-11-01") | (df["day"]=="2021-11-02")]
df_2021_11_03_04 = df[(df["day"]=="2021-11-03") | (df["day"]=="2021-11-04")]

## 1. Read data into bucketed files with one key

In [109]:
clean()
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01, path="s3://crypto-datahub/test_folder", bucketing_info=(["from"],2) ,database="test", table="test_table")

In [110]:
# Une table test_table a été ajouté
wr.catalog.tables()

,Database,Table,Description,TableType,Columns,Partitions
0,test,test_table,,EXTERNAL_TABLE,"blocknumber, timestamp, hash, nonce, blockhash...",


2 fichiers on été crées.

In [117]:
s3_files = list_s3()
print(s3_files)

['s3://crypto-datahub/test_folder/1d48f20c8deb48e5a182f62e1708011e_bucket-00000.snappy.parquet', 's3://crypto-datahub/test_folder/1d48f20c8deb48e5a182f62e1708011e_bucket-00001.snappy.parquet']


In [118]:
file1 = wr.s3.read_parquet(s3_files[0])
file2 = wr.s3.read_parquet(s3_files[1])
from1 = set(file1["from"].values)
from2 = set(file2["from"].values)
print(f"There is {len(from1)} different from address into the file 1")
print(f"There is {len(from2)} different from address into the file 2")
print(f"There is {len(from1.intersection(from2))} address that are both in file 1 and from 2")

There is 157 different from address into the file 1
There is 141 different from address into the file 2
There is 0 address that are both in file 1 and from 2


## 2. Read data into bucketed files with two key

In [119]:
clean()
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01, path="s3://crypto-datahub/test_folder", bucketing_info=(["from","to"],2) ,database="test", table="test_table")

In [120]:
# Une table test_table a été ajouté
wr.catalog.tables()

,Database,Table,Description,TableType,Columns,Partitions
0,test,test_table,,EXTERNAL_TABLE,"blocknumber, timestamp, hash, nonce, blockhash...",


2 fichiers on été crées.

In [121]:
s3_files = list_s3()
print(s3_files)

['s3://crypto-datahub/test_folder/ddc5970d06454f6f9bfa5dc8d6f006c9_bucket-00000.snappy.parquet', 's3://crypto-datahub/test_folder/ddc5970d06454f6f9bfa5dc8d6f006c9_bucket-00001.snappy.parquet']


In [127]:
file1 = wr.s3.read_parquet(s3_files[0])
file2 = wr.s3.read_parquet(s3_files[1])
from1 = set(file1["from"].values)
from2 = set(file2["from"].values)
print(f"There is {len(from1)} different from address into the file 1")
print(f"There is {len(from2)} different from address into the file 2")
print(f"There is {len(from1.intersection(from2))} address that are both in file 1 and file 2")
to1 = set(file1["to"].values)
to2 = set(file2["to"].values)
print(f"There is {len(to1)} different to address into the file 1")
print(f"There is {len(to2)} different to address into the file 2")
print(f"There is {len(to1.intersection(to2))} address that are both in file 1 and file 2")
from_to1 = set(tuple(row) for row in file1[["from","to"]].values)
from_to2 = set(tuple(row) for row in file2[["from","to"]].values)
print(f"There is {len(from_to1)} different from_to address combinaison  into the file 1")
print(f"There is {len(from_to2)} different from_to address combinaison  into the file 2")
print(f"There is {len(from_to1.intersection(from_to2))} address combinaison that are both in file 1 and file 2")

There is 157 different from address into the file 1
There is 142 different from address into the file 2
There is 1 address that are both in file 1 and file 2
There is 7 different to address into the file 1
There is 5 different to address into the file 2
There is 1 address that are both in file 1 and file 2
There is 162 different from_to address combinaison  into the file 1
There is 145 different from_to address combinaison  into the file 2
There is 0 address combinaison that are both in file 1 and file 2


## 3. Write same data twice

In [143]:
clean()
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01, path="s3://crypto-datahub/test_folder", bucketing_info=(["from"],2) ,database="test", table="test_table")
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01, path="s3://crypto-datahub/test_folder", bucketing_info=(["from"],2) ,database="test", table="test_table")

In [144]:
s3_files = list_s3()
print(s3_files)

['s3://crypto-datahub/test_folder/afa2ab48fc8f4aa083fe252e135cc9db_bucket-00000.snappy.parquet', 's3://crypto-datahub/test_folder/afa2ab48fc8f4aa083fe252e135cc9db_bucket-00001.snappy.parquet', 's3://crypto-datahub/test_folder/c362734e7b8a46f6ac746167203b3b37_bucket-00000.snappy.parquet', 's3://crypto-datahub/test_folder/c362734e7b8a46f6ac746167203b3b37_bucket-00001.snappy.parquet']


In [145]:
wr.athena.read_sql_query("SELECT * FROM test_table", database="test")

QueryFailed: HIVE_INVALID_BUCKET_FILES: Hive table 'test.test_table' is corrupt. File 'afa2ab48fc8f4aa083fe252e135cc9db_bucket-00000.snappy.parquet' does not match the standard naming pattern, and the number of files in the directory (4) does not match the declared bucket count (2) for partition: <UNPARTITIONED>. You may need to manually clean the data at location 's3://aws-athena-query-results-408566731358-eu-west-3/tables/70d1afc9-cda1-4337-a3f7-5351f209f448' before retrying. Athena will not delete data in your account.

La requete athena échoue, car comme on a spécifié 2 buckets, le folder devrait contenir seulement 2 fichier mais il en contient 4. Le premier appel créer 2 fichiers, les lignes sont répartis dans les 2 fichier  en fonction de leur address from, et lors du deuxième appelle la même chose se produit, on se retrouve donc avec 4 fichiers dans le même bucket. 

## 4. Change bucketing key

In [183]:
clean()
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01, path="s3://crypto-datahub/test_folder", bucketing_info=(["from"],2) ,database="test", table="test_table")
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01, path="s3://crypto-datahub/test_folder", bucketing_info=(["to"],2) ,database="test", table="test_table")

In [184]:
s3_files = list_s3()
print(s3_files)

['s3://crypto-datahub/test_folder/2757c3cdc7264332a1021d4e6909c8ff_bucket-00000.snappy.parquet', 's3://crypto-datahub/test_folder/2757c3cdc7264332a1021d4e6909c8ff_bucket-00001.snappy.parquet', 's3://crypto-datahub/test_folder/9326dee6079e437197e813e0c36022fb_bucket-00000.snappy.parquet', 's3://crypto-datahub/test_folder/9326dee6079e437197e813e0c36022fb_bucket-00001.snappy.parquet']


In [185]:
wr.athena.read_sql_query("SELECT * FROM test_table", database="test")

QueryFailed: HIVE_INVALID_BUCKET_FILES: Hive table 'test.test_table' is corrupt. File '2757c3cdc7264332a1021d4e6909c8ff_bucket-00000.snappy.parquet' does not match the standard naming pattern, and the number of files in the directory (4) does not match the declared bucket count (2) for partition: <UNPARTITIONED>. You may need to manually clean the data at location 's3://aws-athena-query-results-408566731358-eu-west-3/tables/6e348d58-7ec8-424c-82d1-2ec0065a20ad' before retrying. Athena will not delete data in your account.

Même pb que dans le 3 mais encore pire. La requete athena échoue, car comme on a spécifié 2 buckets, le folder devrait contenir seulement 2 fichier mais il en contient 4. Le premier appel créer 2 fichiers, les lignes sont répartis dans les 2 fichier  en fonction de leur address from, et lors du deuxième appelle la même chose se produit, on se retrouve donc avec 4 fichiers dans le même bucket. 

## 5. Use one partitioning key

In [192]:
clean()
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01_02, partition_cols=["day"], path="s3://crypto-datahub/test_folder", database="test", table="test_table")

In [193]:
s3_files = list_s3()
print(s3_files)

['s3://crypto-datahub/test_folder/day=2021-11-01/da366e47785b4f2e923e90cafb8eef5f.snappy.parquet', 's3://crypto-datahub/test_folder/day=2021-11-02/da366e47785b4f2e923e90cafb8eef5f.snappy.parquet']


In [195]:
print(len(wr.athena.read_sql_query("SELECT * FROM test_table", database="test")))
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01_02, partition_cols=["day"], path="s3://crypto-datahub/test_folder", database="test", table="test_table")
print(len(wr.athena.read_sql_query("SELECT * FROM test_table", database="test")))

1760
3520


overwrite partitions

In [196]:
clean()
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01_02, partition_cols=["day"], path="s3://crypto-datahub/test_folder", database="test", table="test_table")
print(len(wr.athena.read_sql_query("SELECT * FROM test_table", database="test")))
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01_02, partition_cols=["day"], path="s3://crypto-datahub/test_folder", database="test", table="test_table", mode="overwrite_partitions")
print(len(wr.athena.read_sql_query("SELECT * FROM test_table", database="test")))

1760
1760


## 6. Using multiple partitioning key

In [198]:
clean()
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01_02, partition_cols=["day","to"], path="s3://crypto-datahub/test_folder", database="test", table="test_table")

In [ ]:
s3_files = list_s3()
print(s3_files)

['s3://crypto-datahub/test_folder/day=2021-11-01/da366e47785b4f2e923e90cafb8eef5f.snappy.parquet', 's3://crypto-datahub/test_folder/day=2021-11-02/da366e47785b4f2e923e90cafb8eef5f.snappy.parquet']


In [ ]:
print(len(wr.athena.read_sql_query("SELECT * FROM test_table", database="test")))
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01_02, partition_cols=["day"], path="s3://crypto-datahub/test_folder", database="test", table="test_table")
print(len(wr.athena.read_sql_query("SELECT * FROM test_table", database="test")))

1760
3520


overwrite partitions

In [ ]:
clean()
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01_02, partition_cols=["day"], path="s3://crypto-datahub/test_folder", database="test", table="test_table")
print(len(wr.athena.read_sql_query("SELECT * FROM test_table", database="test")))
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01_02, partition_cols=["day"], path="s3://crypto-datahub/test_folder", database="test", table="test_table", mode="overwrite_partitions")
print(len(wr.athena.read_sql_query("SELECT * FROM test_table", database="test")))

1760
1760


## 7. Partitioning then bucketing

In [199]:
clean()
r = wr.s3.to_parquet(dataset=True, df=df_2021_11_01_02, partition_cols=["day"], path="s3://crypto-datahub/test_folder", database="test", table="test_table")

In [201]:
s3_files = list_s3()
print(s3_files)

['s3://crypto-datahub/test_folder/day=2021-11-01/09823586e6d8447b8aecaff08850965d.snappy.parquet', 's3://crypto-datahub/test_folder/day=2021-11-02/09823586e6d8447b8aecaff08850965d.snappy.parquet']


In [231]:
query = ("""
         CREATE TABLE curated_v2 WITH (
format = 'PARQUET', 
external_location = 's3://crypto-datahub/curated', 
partitioned_by = ARRAY['day'], 
bucketed_by = ARRAY['from','to'], 
bucket_count = 100) 
AS SELECT *
FROM  test_table LIMIT 0
""")
r = wr.athena.read_sql_query(query, database="test", ctas_approach=False)

In [242]:
query = ("""
         CREATE TABLE OKOK
    WITH (
      format = 'PARQUET', 
      external_location = 's3://crypto-datahub/curated/day=2021-11-01/', 
      bucketed_by = ARRAY['from','to'], 
      bucket_count = 4) 
    AS SELECT *
    FROM test_table
    WHERE day='2021-11-01';
""")
r = wr.athena.read_sql_query(query, database="test", ctas_approach=False)

QueryFailed: HIVE_PATH_ALREADY_EXISTS: Target directory for table 'test.okok' already exists: s3://crypto-datahub/curated/day=2021-11-01. You may need to manually clean the data at location 's3://aws-athena-query-results-408566731358-eu-west-3/tables/ff40c3b7-589d-4b49-a316-86508aea2968' before retrying. Athena will not delete data in your account.

In [239]:
query = ("""
         ALTER TABLE curated_v2 ADD PARTITION (day='2021-11-01');
""")
r = wr.athena.read_sql_query(query, database="test", ctas_approach=False)